- Python 3.8
- Tensorflow 2.2.0
- `conda install numpy=1.19`

https://stackoverflow.com/questions/66207609/notimplementederror-cannot-convert-a-symbolic-tensor-lstm-2-strided-slice0-t

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
# 28, 28 -> treat image as sequence
# input_size=28
# seq_length=28


# RNN Outputs and states
# ----------------------
# By default, the output of a RNN layer contains a single vector per sample.
# This vector is the RNN cell output corresponding to the last timestep, 
# containing information about the entire input sequence. 
# The shape of this output is (N, units)
# 
# A RNN layer can also return the entire sequence of outputs 
# for each sample (one vector per timestep per sample), 
# if you set return_sequences=True. The shape of this output 
# is (N, timesteps, units).


In [5]:
tf.keras.backend.clear_session()
# model
model = keras.models.Sequential()
model.add(keras.Input(shape=(28,28))) # seq_length, input_size
#model.add(layers.SimpleRNN(128, return_sequences=True, activation='relu')) # N, 28, 128
model.add(layers.LSTM(128, return_sequences=False, activation='relu')) # N, 128
model.add(layers.Dense(10))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               80384     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 81,674
Trainable params: 81,674
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# loss and optimizer
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [7]:
# training
batch_size = 64
epochs = 5

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/5


InternalError:  Blas GEMM launch failed : a.shape=(64, 128), b.shape=(128, 512), m=64, n=512, k=128
	 [[{{node sequential/lstm/while/body/_1/lstm_cell/MatMul_1}}]] [Op:__inference_train_function_1907]

Function call stack:
train_function


In [ ]:
# evaluate
model.evaluate(x_test, y_test, batch_size=batch_size, verbose=2)

157/157 - 1s - loss: 0.0581 - accuracy: 0.9821


[0.058123912662267685, 0.9821000099182129]